<a href="https://colab.research.google.com/github/Jeeva55t/Large_Language_Model-Projects/blob/main/Semantic_Search_and_Question_Answering_using_FAISS%2CSentence_Transformers%2CGroq_LLaMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faiss-cpu  sentence-transformers groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 4.7 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import requests
import os

In [ ]:
GROQ_API_KEY = ""
GROQ_MODEL = "llama3-8b-8192"

In [ ]:
sentences = [
    "I love cats",
    "Dogs are better than cat",
    "AI is the future",
    "Cats are beautiful animal",
    "I enjoy Machine Learning",
    "Snake is dangerous"
]

In [ ]:
embed_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embed_model.encode(sentences).astype('float32')

In [ ]:
embeddings.shape

(6, 384)

In [ ]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

In [ ]:
query ="i am intrested in technology"
query_vector = embed_model.encode([query]).astype("float32")

In [ ]:
top_k = 3
distances,indices = index.search(query_vector ,top_k)

In [ ]:
retrieved_chunks = [sentence[idx] for idx in indices[0]]

In [ ]:
print("Query:", query)
print("\nTop Matches:")
for chunk in retrieved_chunks:
    print("-", chunk)

Query: i am intrested in technology

Top Matches:
- AI is the future
- I enjoy Machine Learning
- I love cats


In [ ]:
context = "\n".join(retrieved_chunks)
prompt =f"""Answer the question based on the context below:
context :
{context}

Question :{query}
Answer:"""



In [ ]:
headers = {
    "Authorization": f"Bearer {GROQ_API_KEY}",
    "Content-Type": "application/json"
}
payload = {
    "model": GROQ_MODEL,
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
}
response = requests.post("https://api.groq.com/openai/v1/chat/completions", headers=headers, json=payload)

In [ ]:
print("\nTop Matches:")
for chunk in retrieved_chunks:
    print("-", chunk)

print("\nFinal Answer:\n", response.json()["choices"][0]["message"]["content"])


Top Matches:
- AI is the future
- I enjoy Machine Learning
- I love cats

Final Answer:
 It sounds like you're interested in technology! Well, since you enjoy Machine Learning, I think you'll be excited to know that it's a crucial part of AI, which is the future! Who knows, maybe one day AI will even help us create the purr-fect cat detection algorithm. Wouldn't that be something?
